# Rankings

### top vs takeOrdered

In [10]:
products = sc.textFile("retail_db/products") 
filteredProducts = products.filter(lambda p: p.split(",")[4] != "")

#for i in filteredProducts.take(3): print i

#top will give data by default in descending order 
topNProducts =  filteredProducts.top(5, key = lambda k: float(k.split(",")[4] ) )   

#negate the key value to get items in acsending order
#topNProducts =  filteredProducts.top(5, key = lambda k: -float(k.split(",")[4] ) )   

for i in topNProducts: print i

#takeordered will give data by default in ascending order    
takeOrderedProducts =  filteredProducts.takeOrdered(2, key = lambda k: float(k.split(",")[4] ) )   

#negate the key value to get items in descending order
#takeOrderedProducts =  filteredProducts.takeOrdered(5, key = lambda k: -float(k.split(",")[4] ) )   

for i in takeOrderedProducts: print i    

208,10,SOLE E35 Elliptical,,1999.99,http://images.acmesports.sports/SOLE+E35+Elliptical
66,4,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill
199,10,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill
496,22,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill
1048,47,"Spalding Beast 60"" Glass Portable Basketball ",,1099.99,http://images.acmesports.sports/Spalding+Beast+60%22+Glass+Portable+Basketball+Hoop
38,3,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat
388,18,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat


### Get top N products by price with in each category - By key Ranking -groupByKey and flatMap

In [15]:
products = sc.textFile("retail_db/products") 
filteredProducts = products.filter(lambda p: p.split(",")[4] != "")

productsMap = filteredProducts.map \
              (lambda p: (int(p.split(",")[1]) , p))

productsGroupByCategoryId = productsMap.groupByKey();

for i in productsGroupByCategoryId.take(2):print i

topNProductsByCategory = productsGroupByCategoryId. \
flatMap(lambda p: sorted(p[1],key = lambda k: float(k.split(",")[4]) , reverse=True)[:3])

for i in topNProductsByCategory.take(1):print i


(2, <pyspark.resultiterable.ResultIterable object at 0x7ff21796ddd0>)
(4, <pyspark.resultiterable.ResultIterable object at 0x7ff21796d710>)
16,2,Riddell Youth 360 Custom Football Helmet,,299.99,http://images.acmesports.sports/Riddell+Youth+360+Custom+Football+Helmet


### Get all top N priced products by category 

In [76]:
products = sc.textFile("retail_db/products") 
filteredProducts = products.filter(lambda p: p.split(",")[4] != "")

productsMap = filteredProducts.map \
              (lambda p: (int(p.split(",")[1]) , p))

productsGroupByCategoryId = productsMap.groupByKey();



def getTopNPricedProduct(productsGroupByCategory , topN):
    sortedProducts = sorted(productsGroupByCategory[1],key=lambda k: float(k.split(",")[4]),reverse=True )
    sortedProductsMap = map(lambda m:float(m.split(",")[4]),sortedProducts)
    topNPrices = sorted(set(sortedProductsMap),reverse=True )[:3]
    import itertools as it
    return it.takewhile(lambda k: float(k.split(",")[4]) in topNPrices, sortedProducts )

#test function with single value
#t = productsGroupByCategoryId.filter(lambda p: p[0] == 59).first()
#result = list(getTopNPricedProduct(t,3))
#for i in result:
#    print i

topNPricedProduct = productsGroupByCategoryId.flatMap(lambda p: getTopNPricedProduct(p,3))

#for i in topNPricedProduct.take(100):print i

#help(it)
#help(it.takewhile)

 
#help(sorted)


1323,59,"Nike Men's Home Game Jersey Cleveland Browns ",,100.0,http://images.acmesports.sports/Nike+Men%27s+Home+Game+Jersey+Cleveland+Browns+Johnny+Manziel...
1330,59,Nike Men's Home Game Jersey Denver Broncos Pe,,100.0,http://images.acmesports.sports/Nike+Men%27s+Home+Game+Jersey+Denver+Broncos+Peyton+Manning...
1331,59,Nike Men's Home Game Jersey Buffalo Bills Sam,,100.0,http://images.acmesports.sports/Nike+Men%27s+Home+Game+Jersey+Buffalo+Bills+Sammy+Watkins+%2314
1332,59,Nike Men's Home Game Jersey San Francisco 49e,,100.0,http://images.acmesports.sports/Nike+Men%27s+Home+Game+Jersey+San+Francisco+49ers+Colin...
1333,59,Nike Men's Home Game Jersey Pittsburgh Steele,,100.0,http://images.acmesports.sports/Nike+Men%27s+Home+Game+Jersey+Pittsburgh+Steelers+Ryan+Shazier...
1334,59,Nike Men's Home Game Jersey New Orleans Saint,,100.0,http://images.acmesports.sports/Nike+Men%27s+Home+Game+Jersey+New+Orleans+Saints+Brandin+Cooks...
1335,59,Nike Men's Andrew Luck Jersey - Home Game Ind,,10